In [1]:
import cv2 #สำหรับการจัดการรูปภาพและvdo
import numpy as np #ที่ใช้ในการคำนวนทางคณิตศาสตร์
import pandas as pd #จัดการdataframe

import matplotlib.pyplot as plot
%matplotlib inline

**Import tensorflow libraries**

In [ ]:
import numpy as np #ที่ใช้ในการคำนวนทางคณิตศาสตร์
import pandas as pd #จัดการdataframe

import matplotlib.pyplot as plot
%matplotlib inline

In [ ]:
import tensorflow #สำหรับทำ machine learning models
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os #ใช้จัดการไฟล์
os.listdir("../input/")

ดึงข้อมูลรูปภาพจากtraining set directory

In [ ]:
def get_images(directory):
    Images = []
    Labels = []
    for dir_name in os.listdir(directory): 
        for image_file in os.listdir(directory+dir_name):
            image = cv2.imread(directory+dir_name+r'/'+image_file)
            if image is not None:
                image = cv2.resize(image,(300,300),)
                Images.append(image)
                Labels.append(dir_name)
    return Images, Labels

In [ ]:
Images, Labels = get_images('../input/train/')

เปลี่ยนข้อมูลจากตัวอักษรไปเป็นตัวเลข เพราะmachine learning model เข้าใจแต่ตัวเลข

In [ ]:
labels = []
mapping = { 'Sugar beet': 0, 'Fat Hen': 1, 'Scentless Mayweed' : 2, 'Charlock' : 3,
           'Small-flowered Cranesbill': 4, 'Maize': 5, 'Shepherds Purse' :6, 'Common wheat': 7,
           'Common Chickweed': 8, 'Cleavers': 9, 'Loose Silky-bent' : 10, 'Black-grass': 11 }
for label in Labels:
    labels.append(mapping[label])
del Labels

In [ ]:
Images[0].shape

เปลี่ยนข้อมูลให้เป็น 4 มิติเพื่อให้โมเดลสามารถนำไปใช้ได้ (No. of images, height, width, channels)

In [ ]:
Images = np.reshape(Images,(-1,300,300,3))
Labels = np.array(labels)

In [ ]:
print("Shape of training data: ", Images.shape)
print("Shape of labels data: ", Labels.shape)

splitting ข้อมูลเป็น Training และ Validation เพื่อเช็คความแม่นยำของโมเดลบนโมเดลที่ยังไม่เคยเห็น

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(Images, Labels, test_size=.2, random_state=42, stratify = Labels)

In [ ]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,num_classes=12)
y_val = np_utils.to_categorical(y_val,num_classes=12)

ImageDataGenerator ช่วยในเรื่องของ image augmentation โดยการดำเนินการต่างๆบนรูปภาพที่มีอยู่

In [ ]:
train_datagen = ImageDataGenerator(
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                  )

validation_datagen = ImageDataGenerator()

In [ ]:
del Images
del Labels

In [ ]:
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
validation_generator = validation_datagen.flow(x_val, y_val, batch_size=16)

Pretrain โมเดลด้วย VGG16(เป็นการโครงสร้างCNNที่มีคนทำไว้อยู่แล้วมาใช้แล้วค่อยไปปรับแต่งทีหลังเพื่อที่จะได้ไม่ต้องเสียเวลาไปเทรนเอง)

In [ ]:
from tensorflow.keras.applications import VGG16
vgg = VGG16(include_top=
            False, weights='imagenet', input_shape = (300,300,3))

In [ ]:
import tensorflow.keras.optimizers as Optimizer
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAvgPool2D, GlobalMaxPooling2D, Concatenate
from tensorflow.keras.models import Model

fix base model เอาไว้ แล้วเปลี่ยนแค่ส่วน head(ส่วนclassification) ให้เป็น custom head ที่เราทำเอง

In [ ]:
vgg.trainable=False
for layer in vgg.layers:
    layer.trainable = False

fc1 = Concatenate(axis=-1)([GlobalAvgPool2D()(vgg.output), GlobalMaxPooling2D()(vgg.output)])
fc1 = Dense(400, activation='relu')(fc1)
fc1_dropout = Dropout(0.3)(fc1)
fc2 = Dense(200, activation='relu')(fc1_dropout)
fc2_dropout = Dropout(0.3)(fc2)
fc2 = Dense(75, activation='relu')(fc1_dropout)
output = Dense(12, activation='softmax')(fc2_dropout)
model = Model(vgg.input, output)

model.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


ในระหว่างที่เทรนเซฟ model weights ที่ดีที่สุดด้วย ModelCheckpoint

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('saved_model.hdf5', monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
trained = model.fit_generator(train_generator,steps_per_epoch = 25, epochs=200, validation_data = validation_generator,
                              validation_steps=10, 
                              verbose=1, callbacks = callbacks_list)

plotกราฟของค่า accuracy และ loss

In [ ]:
plot.plot(trained.history['acc'])
plot.plot(trained.history['val_acc'])
plot.title('Model accuracy')
plot.ylabel('Accuracy')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

plot.plot(trained.history['loss'])
plot.plot(trained.history['val_loss'])
plot.title('Model loss')
plot.ylabel('Loss')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

In [ ]:
def get_test_images(directory):
    Images = []
    Image_names = []
    for image_file in os.listdir(directory):
        Image_names.append(image_file)
        image = cv2.imread(directory+r'/'+image_file)
        if image is not None:
            image = cv2.resize(image,(300,300),)
            Images.append(image)
    return Images, Image_names

In [ ]:
test_images, image_names = get_test_images('../input/test/')
test_images = np.array(test_images)
print(test_images.shape)

เพื่อที่จะโหลด weights ที่ save ไว้เราจะต้องกำหนด model architecture เดิมอีกครั้ง

In [ ]:
vgg = VGG16(include_top=
            False, weights='imagenet', input_shape = (300,300,3))

vgg.trainable=False
for layer in vgg.layers:
    layer.trainable = False

fc1 = Concatenate(axis=-1)([GlobalAvgPool2D()(vgg.output), GlobalMaxPooling2D()(vgg.output)])
fc1 = Dense(400, activation='relu')(fc1)
fc1_dropout = Dropout(0.3)(fc1)
fc2 = Dense(200, activation='relu')(fc1_dropout)
fc2_dropout = Dropout(0.3)(fc2)
fc2 = Dense(75, activation='relu')(fc1_dropout)
output = Dense(12, activation='softmax')(fc2_dropout)
model = Model(vgg.input, output)

In [ ]:
model.load_weights('saved_model.hdf5')

saveโมเดลด้วย tensorflow.keras.models.save_model()

In [ ]:
tensorflow.keras.models.save_model(
    model,
    'tf_model.hdf5',
    overwrite=True,
    include_optimizer=True
)

จากนั้นเราสามารถโหลดทั้งโมเดลได้ด้วย tensorflow.keras.models

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('tf_model.hdf5')

จากนั้นโมเดลก็สามารถ predicts ภาพใหม่ได้ด้วยฟังก์ชั่น model.predict()

In [ ]:
predictions = model.predict(test_images)
predictions = np.argmax(predictions, axis = 1)

In [ ]:
labelled_predictions = []
mapping = {0: 'Sugar beet',1:'Fat Hen' ,2: 'Scentless Mayweed',3:  'Charlock', 
        4:'Small-flowered Cranesbill', 5:'Maize' ,
        6: 'Shepherds Purse' ,7:'Common wheat' ,8:'Common Chickweed' ,
        9:'Cleavers' ,10:'Loose Silky-bent'  ,11: 'Black-grass'}
for pred in predictions:
    labelled_predictions.append(mapping[pred])

เตรียมข้อมูลการ Predict สำหรับใช้ submit

In [ ]:
d = []
i=0
for pred in labelled_predictions:
    d.append({'file': image_names[i], 'species': pred})
    i=i+1
output = pd.DataFrame(d)
output.to_csv('submission.csv',index=False)

#  Conclusion

 จากที่เราได้ลองทำ Image classification ด้วยการทำ Transfer  learning จาก VGG16 และเทรนไป 200 epochs ได้ผลลัพท์ F1 scoreที่ 0.87 ซึ่งถือว่าสามารถนำไปใช้งานได้และค่อนข้างแม่นยำ อีกทั้งยังสามารถเพิ่มความสามารถไปอีกได้ด้วยการลองเปลี่ยนไปใช้ Model ต้นแบบอื่น หรืออาจจะเพิ่มจำนวน Epoch ขึ้น

# Reference

* Step by step VGG16 implementation in Keras for beginners , Rohit Thakur. https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c 
* Transfer Learning with Keras for Computer Vision Applications , nuttachot promrit. https://blog.pjjop.org/transfer-learning-with-keras-for-computer-vision-applications/
* Plant Seedling Classification , SAICHAND. https://www.kaggle.com/code/dsaichand3/plant-seedling-classification

# สมาชิก

* ทัศน์พล แต้มช่วย 6424650056
* วรดนัย วงษ์มณี 6424650189
* นายธีรวิสิทธ์ จินนารัตน์ 6424650312
* นายวิชัย มาเจริญ 6424650379
* ศิรวิทย์ ตันติกุล 6424650387
* ศิวกร นิตย์กิจสมบูรณ์ 6424650395

